# Dataset Generator

We need to generate a dataset of reflections. To do this we need to gather some data from the internet. Make sure you have already executed the appropriate cells in Cepbimo - Preamble.

## Data sources

### Anechoic Data

In order to generate a dataset of reflections we need to start with anechoic data. A dataset of anechoic orchestral recordings is available thanks to [(Pätynen and Lokki, 2008)](#patynen-lokki-08).
***
> 1. [Mozart's An aria of Donna Elvira](https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/mozart_mp3.zip)
>   - Flute, clarinet, bassoon, french horns (1-2), violin (I), violin (II), viola, cello, contrabass, soprano (soloist)
> 2. [Beethoven's Symphony no. 7, I movement, bars 1-53](https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/beethoven_mp3.zip)
>   - Flutes (1-2), oboes (1-2), clarinets (1-2), bassoon (1-2), french horns (1-2),
>     trumpets (1-3), timpani, violin (I), violin (II), viola, cello, contrabass
> 3. [Bruckner's Symphony no. 8, II movement, bars 1-61](https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/bruckner_mp3.zip)
>   - Flutes (1-3), oboes (1-3), clarinets (1-3), bassoon (1-3),
>     french horns (1-8), trumpets (1-3), trombones (1-3), tuba, timpani,
>     violin (I) (two divisi), violin (II) (two divisi), viola (two divisi),
>     cello (two divisi), contrabass (two divisi)
> 4. [Mahler's Symphony no. 1, IV movement, bars 1-85](https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/mahler_mp3.zip)
>   - Piccolo (1-2) (fl1), flutes (1-2) (fl3), oboes (1-4), clarinets (1-4),
> bassoon (1-3), french horns (1-7), trumpets (1-4), tuba, timpani,
> percussions (1-2), violin I (two divisi), violin II (two divisi),
> viola, cello, contrabass
***
For more information on the anechoic data we use click [this link.](https://research.cs.aalto.fi//acoustics/virtual-acoustics/research/acoustic-measurement-and-analysis/85-anechoic-recordings.html)

In [ ]:
# @title Anechoic Data
# @markdown Anechoic data recorded by [(Pätynen and Lokki, 2008)](#patynen-lokki-08)
# @markdown > Links to anechoic recordings
file_links = [
              'https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/beethoven_mp3.zip',
              'https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/mozart_mp3.zip',
              'https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/bruckner_mp3.zip',
              'https://mediatech.aalto.fi/images/research/virtualacoustics/recordings/mahler_mp3.zip']
# @markdown > Download files if they don't exist
for l in file_links:
  ! wget -nc {l}

### Head-related Transfer Functions

Next we need a dataset of head-related transfer functions (HRTF). A set of HRTFs is available thanks to [(Gardner & Martin, 1994)](#gardner-martin-94) and MIT Media Lab.
***
For more information see [KEMAR HRTF measurements](https://sound.media.mit.edu/resources/KEMAR.html)

In [ ]:
# @title HRTF Data
# @markdown HRTF data recorded by [(Gardner & Martin, 1994)](#gardner-martin-94)
! wget -nc https://sound.media.mit.edu/resources/KEMAR/full.zip

### Extracting Data

Extract the archived data to the `data/` directory.
***
**Warning: Overwrites existing files**

In [ ]:
# @title Extracting Data
# @markdown Extract zipped data to `data/` directory.
# @markdown > List of files to unzip
zip_files = [
             ['beethoven_mp3.zip', 'data/anechoic/beethoven'],
             ['mozart_mp3.zip', 'data/anechoic/mozart'],
             ['bruckner_mp3.zip', 'data/anechoic/bruckner'],
             ['mahler_mp3.zip', 'data/anechoic/mahler'],
             ['full.zip', 'data/hrtfs']]
# > Unzip each archive to `data/anechoic` directory
for z in zip_files:
  !unzip -o {z[0]} -d {z[1]}

## Dataset Parameters

### Random seed

Pseudo-random number generators (RNG) use seed values to produce sequences of random numbers. By using a seed value we are able to generate a reproducible random dataset. The seed value we use is a hexidecimal number between 0x000000 and 0xffffff. The default seed value can be changed using the slider below.

In [21]:
import interactive
from notebook import *
import matplotlib.pyplot as plt
_ = plt.ion();

make_seed_picker()

### Sample duration

The anechoic data is longer than necessary. The sample duration has a default value of 20 and determines the length of each sample in seconds. Use the slider below to change the default value.

In [23]:
make_duration_picker()

### HRTF

Head-related transfer functions (HRTF) describe the relationship between the direction a person is looking and the perceived direction of a sound in terms of zenith, or elevation, and azimuth. By default HRTFs range in zenith from -30$^{\circ}$ to 90$^{\circ}$ and azimuths range from 0$^{\circ}$ to 360$^{\circ}$. Use the sliders below to adjust the zenith and azimuth ranges.

In [4]:
make_hrtf_picker()

### Reflections

A reflection is defined by amplitude, delay, zenith, and azimuth. The zenith and azimuth were set in the HRTF section while the amplitude is set to the range 0-1. By default there will be between 4-8 reflections per sample with default delays in the range 1-60ms. The reflection count and delay ranges can be adjusted below.

In [5]:
make_reflection_picker()

### Reverberation

Reverberation is defined by amplitude, delay, and time. Reverberation and refelction share the properties delay and amplitude while time represents the duration of reverberation in seconds. By default reverberation can have a duration between 1-8s. To change the reverberation time adjust the slider below

In [6]:
make_reverberation_picker()

### Sample count

Lastly we need to determine how many samples to generate. You can generate up to 10 samples by adjusting the slider.

In [7]:
make_count_picker()

### Review parameters

Review the parameters and the sample data produced. To edit parameters adjust the appropriate slider then rerun the cell below to update review. When you are ready press the `generate` button.

In [8]:
make_reviewer()

### Generate dataset

In [32]:
def generate_data():
    from data_generator import DataGenerator
    from RNG import RNG
    
    rng = RNG(
        seed = dataset_parameters['seed'],
        duration = dataset_parameters['duration'],
        delay_limits = dataset_parameters['delay_limits'],
        time_limits = dataset_parameters['time_limits'],
        reflection_limits= dataset_parameters['reflection_limits'],
        zenith_limits = dataset_parameters['zenith_limits'],
        azimuth_limits = dataset_parameters['azimuth_limits'])
    
    dg = DataGenerator(dataset_parameters['count'], rng).generate()
    return dg
    
    
df = generate_data()
df

Start plotting samples:
Finished plotting samples:

Start mixing parts into signals:
Finished mixing parts into signals:

Start plotting signals:
Finished plotting signals:

Start applying HRTFs:
Finished applying HRTFs:

Start plotting HRTFs:
Finished plotting HRTFs:

Start applying reflections:
Finished applying reflections:

Start plotting reflections:
Finished plotting reflections:

Start applying reverberation:
Finished applying reverberation:

Start plotting reverberation:
Finished plotting reverberation:

Start summing signals:
Finished summing signals:

Start plotting summation:
Finished plotting summation:

Start adjusting signal-to-noise ratio:
Finished adjusting signal-to-noise ratio:

Start plotting noise:
Finished plotting noise:

Start trimming samples:
Finished trimming samples:

Start plotting trimmed:
Finished plotting trimmed:

Start plotting cepstrums:
Finished plotting cepstrums:

Start plotting RIR:
Finished plotting RIR:

Start generating Cepbimo:
Finished generat

,zenith,azimuth,composer,part_count,parts,offset,duration,reverb_time,reverb_delay,reverb_amplitude,...,reflection_zenith,reflection_azimuth,filepath,signal,reflections,reverberation,summation,noise,sample,cepbimo
0,50,80,mahler,20,"[data\anechoic\mahler\mahler_corno2_6.mp3, dat...",105937,20,3,37,0.029793,...,"[60, 10, 30, 50]","[40, 270, 72, 256]",mahler_p20_a080_e050,(((<pydub.audio_segment.AudioSegment object at...,(((<pydub.audio_segment.AudioSegment object at...,(((<pydub.audio_segment.AudioSegment object at...,(((<pydub.audio_segment.AudioSegment object at...,(((<pydub.audio_segment.AudioSegment object at...,(((<pydub.audio_segment.AudioSegment object at...,<Cepbimo.Cepbimo object at 0x0000025DD5E2CFA0>


## References

* Gardner, B., & Martin, K. (1994). Hrtf measurements of a kemar dummy-head microphone. MIT Media Lab. Perceptual Computing-Technical Report, 280, 1-7.
<a name='gardner-martin-94'><a/>

* Pätynen, J., Pulkki, V., & Lokki, T. (2008). Anechoic recording system for symphony orchestra. Acta Acustica united with Acustica, 94(6), 856-865. 
<a name='patynen-lokki-08'><a/>